<a href="https://colab.research.google.com/github/Bhardwaj-Saurabh/Agents_Design_Pattern/blob/master/MultiAgent_Pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install openai

In [2]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [3]:
from openai import OpenAI

In [4]:
client = OpenAI()

In [5]:
EASSY_SYSTEM_PROMPT = "You are an eassy writer which writes essay in british english."

In [7]:
TRANSLATOR_SYSTEM_PROMPT = "You are an english to telugu translator. You are given the english essay below. \n %s. \n.Translate it to telugu language"

In [8]:
def agent(system_prompt, user_query):
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_query}
            ]
        )

    return response.choices[0].message.content

In [9]:
def multiagent_pipeline(user_query: str):

    state_dict = dict()

    essay = agent(EASSY_SYSTEM_PROMPT, user_query)
    state_dict['english_essay'] = essay

    if state_dict['english_essay']:
        translation = agent(TRANSLATOR_SYSTEM_PROMPT, state_dict['english_essay'])
        state_dict['telugu_essay'] = translation

        if state_dict['telugu_essay']:
            with open('telugu_essay.txt', 'w+') as f:
                f.write(state_dict['telugu_essay'])
        else:
            print('error translating english essay')
    else:
        print('error while generating english essay')

    return state_dict

In [10]:
state_dict = multiagent_pipeline('write an essay about python snake.')

In [11]:
state_dict['english_essay']

'**The Fascinating World of Python Snakes**\n\nThe Python snake, a remarkable member of the reptilian family, has captivated the imagination of humans for centuries. These large, non-venomous constrictors are renowned for their impressive size, striking patterns, and unique behaviours. This essay will explore the characteristics, habitat, diet, and ecological significance of Python snakes, delving into why they are not only intriguing creatures but also essential to the ecosystems they inhabit.\n\n**Characteristics of Python Snakes**\n\nPythons belong to the family Pythonidae and are found primarily in the tropics and subtropics of Africa, Asia, and Australia. They exhibit a range of sizes; some species, such as the reticulated python (Malayopython reticulatus), can grow over six metres long, making them one of the longest snake species in the world. They have a stout body covered in scales that display intricate patterns, which serve as camouflage in their natural habitats. The colora

In [13]:
state_dict['telugu_essay']

'**పైథాన్ పాముల కక్ష్యా లోని ఆకర్షణీయమైన ప్రపంచం**\n\nపైథాన్ పాము, రాణీయుల కుటుంబానికి చెందిన అద్భుతమైన సభ్యుడు, శతాబ్దాలుగా మానవుల ఊహలను ఆకర్షించనుంది. ఈ పెద్ద, విషస్రావం లేని కంకాళాలు తమ ప్రత్యేక ఆకారానికి, ఆకర్షణీయ నిధీకి మరియు అసాధారణ ప్రవర్తనకు పేరుగాంచాయి. ఈ వ్యాసంలో, పాండిత్యం, నివాసం, ఆహారం మరియు పర్యావరణ సంబంధిత పాత్రను పరిశీలించడమే కాక, ఇతని ఆకర్షణీయ వ్యక్తిత్వానికి మరియు నివసించే పర్యావరణాలకు అవసరమైనదిగా పాండిత్యం ఎందుకు ఉంది అనేది వివరిస్తుంది.\n\n**పైథాన్ పాముల లక్షణాలు**\n\nపైథాన్ పాములు పైనాయిడా కుటుంబానికి సంబంధించినవి మరియు అనుకుని ఆఫ్రికా, ఆసియా మరియు ఆస్ట్రేలియా త్రోపికల్ మరియు సబ్-త్రోపికల్ ప్రాంతాలలో ఎక్కువగా కనిపిస్తాయి. ఈ పాముల పరిమాణంలో విస్తృత శ్రేణి ఉంది; రెటికులేటెడ్ పైథాన్ (మలయోపీథాన్ రెటికులటస్) వంటి కొన్ని జాతులు ఆరు మీటర్ల పైకి పెరుగుతాయి, ఇవి ప్రపంచంలోని అత్యంత పొడవైన పాము జాతులలో ఒకటిగా ఉంటాయి. వాటి బలం ఉన్న శరీరాలు కంచె చమురు ద్వారా కప్పబడ్డాయి, ఇవి స్వాభావిక నివాసాలలో కమ్ము చేయడానికి ఉపయోగిస్తాయి. రంగులు భూమి మెడలు మరియు ఆకుండలు వంటి మిశ్రమ రంగుల నుండ

In [14]:
class OpenAIAgentChained:
    def __init__(self, func):
        self.func = func
    def __call__(self, input_text):
        return self.func(input_text)
    # Add __rshift__ method to allow chaining OpenAIAgentChained objects
    def __rshift__(self, other_agent):
        def chained_function(input_text):
            # Call the current chained function first
            intermediate_result = self(input_text)
            # Then call the next agent with the intermediate result
            return other_agent(intermediate_result)
        # Return a new OpenAIAgentChained object with the combined function
        return OpenAIAgentChained(chained_function)

class OpenAIAgent:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
        self.client = OpenAI()

    def __call__(self, input_text):
        import openai
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": self.prompt_template},
                {"role": "user", "content": input_text}
            ]
        )
        return response.choices[0].message.content

    def __rshift__(self, other_agent):
        def chained_function(input_text):
            result = self(input_text)
            return other_agent(result)
        return OpenAIAgentChained(chained_function)


In [15]:
from pathlib import Path

class SaveToFileAgent:
    def __init__(self, filepath: str):
        self.filepath = Path(filepath)

    def __call__(self, content: str) -> str:
        self.filepath.write_text(content, encoding="utf-8")
        return f"✅ Output saved to {self.filepath.resolve()}"

    def __rshift__(self, other_agent):
        def chained_func(user_input: str):
            output = self(user_input)
            return other_agent(output)
        return OpenAIAgentChained(chained_func)

In [16]:
eassy_write = OpenAIAgent(EASSY_SYSTEM_PROMPT)
translator = OpenAIAgent(TRANSLATOR_SYSTEM_PROMPT)
savefileagent = SaveToFileAgent('/content/eassy.txt')


pipeline = eassy_write >> translator >> savefileagent

In [21]:
result = pipeline('write an eassy about about Bhagavat geeta 18 chapters.')

In [18]:
from IPython.display import Markdown
Markdown(result)

✅ Output saved to /content/eassy.txt